Connected to .venv (Python 3.13.2)

Load all environment variables from .env file

In [2]:
%load_ext dotenv
%dotenv

Controlflow needs Prefect server. Prefect needs sqlite to be installed.

In [26]:
# Verify Prefect API is accessible
import httpx

try:
    response = httpx.get("http://localhost:4200/api/health")
    response.raise_for_status()
    print("Prefect API is accessible!")
except httpx.HTTPStatusError as e:
    print(f"Prefect API is not accessible: {e}")

Prefect API is accessible!


In [24]:
emails: list[str] = [
    "Hello, I need an update on the project status.",
    "Subject: Exclusive offer just for you!",
    "Urgent: Project deadline moved up by one week.",
]

In [ ]:
import controlflow as cf

# Create a ControlFlow task to generate an reply
reply = cf.run(
    "Write a polite reply to an email",
    context=dict(email=emails[0]),
)

print(reply)

In [ ]:
import controlflow as cf


# Create agents
classifier = cf.Agent(
    name="Email Classifier",
    model="openai/gpt-4o-mini",
    instructions="You are an expert at quickly classifying emails. Always "
                 "respond with exactly one word: either 'important' or 'spam'."
)

responder = cf.Agent(
    name="Email Responder",
    model="openai/gpt-4o",
    instructions="You are an expert at crafting professional email responses. "
                 "Your replies should be concise but friendly."
)


# Create the flow
@cf.flow
def process_email(email_content: str):

    # Classify the email
    category = cf.run(
        f"Classify this email",
        result_type=["important", "spam"],
        agents=[classifier],
        context=dict(email=email_content),
    )

    # If the email is important, write a response
    if category == "important":
        response = cf.run(
            f"Write a response to this important email",
            result_type=str,
            agents=[responder],
            context=dict(email=email_content),
        )
        return response

    # Otherwise, no response is needed
    else:
        print("No response needed for spam email.")


# Run the flow on each email
for email in emails:
    response = process_email(email)
    print(response)